In [43]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sktime.datatypes._panel._convert import from_2d_array_to_nested, from_nested_to_2d_array
from sktime.classification.kernel_based import RocketClassifier

pd.options.mode.chained_assignment = None

In [44]:
Train = pd.read_csv("content/train_dataset_train.csv")
Test = pd.read_csv("content/test_dataset_test.csv")

In [45]:
Train = Train[Train.columns.sort_values()]
Train = Train.set_index('id')
Train = Train.drop([".geo","area"], axis = 1)

In [46]:
Test = Test[Test.columns.sort_values()]
Test = Test.set_index('id')
Test = Test.drop([".geo","area"], axis = 1)

In [47]:
X = Train.drop('crop', axis =1)
X = from_2d_array_to_nested(X, index = X.index)
y = Train['crop'].astype('category')
Test = from_2d_array_to_nested(Test, index = Test.index)

In [48]:
classifier = RocketClassifier()

In [49]:
%%time
classifier.fit(X, y)

CPU times: user 1min 41s, sys: 16.1 s, total: 1min 57s
Wall time: 43.7 s


RocketClassifier()

In [50]:
y_pred = classifier.predict(Test)

In [51]:
res = Test.drop(0, axis = 1).join(pd.DataFrame(y_pred).set_index(Test.index)).rename(columns = {0 : 'crop'})

### ------------------------------------------------------------------------------------------

In [52]:
Test = pd.read_csv("content/test_dataset_test.csv")
Test = Test[Test.columns.sort_values()]
Test = Test.set_index('id')
Test = Test.drop([".geo","area"], axis = 1)

In [53]:
Test = Test.join(res)

In [54]:
col = Train.columns.to_list()
col = col[1:]
col.append('crop')
Test.columns = col

In [55]:
data = pd.concat([Train,Test],axis = 0)

In [56]:
for i in range(7):
    for j in range(1,71):
        data.iloc[:,j][(data.iloc[:,j] <= 0.05) & (data['crop'] == i)] = data.iloc[:,j][(data.iloc[:,j] > 0.05) & (data['crop'] == i)].mean()

In [57]:
Test = data.loc[Test.index].drop('crop', axis = 1)
Train = data.loc[Train.index]

In [58]:
X = Train.drop('crop', axis = 1)
y = Train['crop'].astype('category')

In [59]:
rfc = RandomForestClassifier()

In [60]:
%%time
rfc.fit(X, y)

CPU times: user 1.17 s, sys: 0 ns, total: 1.17 s
Wall time: 1.17 s


RandomForestClassifier()

In [61]:
y_pred = rfc.predict(Test)

In [62]:
test = pd.read_csv("content/test_dataset_test.csv")

In [63]:
test.join(pd.DataFrame(y_pred)).set_index('id').rename(columns = {0 : 'crop'})['crop'].to_csv('rf.csv')